In [ ]:
# This notebook populates, calculates, and concatenates NPH information using a .csv file containing NPH sample info obtained from the Core
# The output is a .txt file that will be uploaded into the respective Qiita Study sample information metadata file
# MAKE A COPY OF THIS NOTEBOOK TO USE FOR EACH SAMPLE SHIPMENT
# MAKE SURE LOT INFORMATION IS UPDATED AND ACCURATE - MACKENZIE WILL HAVE THIS

In [ ]:
# TERMS of metadata columns:
# sample: raw sample, ie stool
# storage liquid: storage liquid used with sample, ie DNA/RNA shield
# storage tube: tube used to store sample and storage liquid - storage tube also includes labels
# homogenate: sample + storage iquid, homogenously combined
# aliquot: refers to the process of aliquoting material into the matrix tube for extraction

In [ ]:
# Import pandas
import pandas as pd

In [ ]:
# Step 1: Import .csv file from Core and check output is as expected:

In [ ]:
# Import and define df
calc_df = pd.read_csv('/Users/mmbryant/Documents/Python Things/Initial Attempts/TEMPLATE Sample Processing spreadsheet notebook test.csv')

In [ ]:
# Check that calc_df looks as expected
calc_df.head()

In [ ]:
# List column names that are currently present in calc_df
list(calc_df.columns)

In [ ]:
# Check for tube_id and/or sample_name columns are present in file (this might change depending on what col name we move forward with)
if 'tube_id' in calc_df.columns or 'sample_name' in calc_df.columns:
   print("tube_id and/or sample_name column is present in file : OK to proceed")
else:
   print("tube_id and/or sample_name column NOT present in file : DO NOT to proceed")

In [ ]:
# Step 2: Confirm lot #, add lot number specific columns and remaining information, and check that all values required for calculations are present

In [ ]:
# Check that the current lot number is as expected before adding lot specific information to df

# Current info is for lot #: 219839  !! (Oct 2023)

calc_df['storage_liquid_lot_number'].value_counts()

In [ ]:
# Add lot number specific information to calc_df
# REMINDER: Current info is for lot # 219839  !!
calc_df['mass_storage_tube_and_storage_liquid_before_sample_g']='16.9'
calc_df['mass_storage_tube_and_storage_liquid_before_sample_mg']='16900'
calc_df['density_storage_liquid_kg_l']='1.11'
calc_df['density_storage_liquid_g_ml']='1.11'
calc_df['mass_storage_tube_only_g']='7.18' #this won't change for NPH, = shield tube + label
calc_df['density_sample_g_ml']='1.06' #this won't change for NPH, = stool
calc_df['density_sample_kg_l']='1.06' #this won't change for NPH, = stool
calc_df['storage_liquid_type']='DNA/RNA Shield' #this won't change for NPH

In [ ]:
# Change data type of new values (was having issues with calculation due to data type - can this function be concatenated with above?)
calc_df['mass_storage_tube_and_storage_liquid_before_sample_g'] = calc_df['mass_storage_tube_and_storage_liquid_before_sample_g'].astype('float')
calc_df['mass_storage_tube_and_storage_liquid_before_sample_mg'] = calc_df['mass_storage_tube_and_storage_liquid_before_sample_mg'].astype('float')
calc_df['density_storage_liquid_kg_l'] = calc_df['density_storage_liquid_kg_l'].astype('float')
calc_df['density_storage_liquid_g_ml'] = calc_df['density_storage_liquid_g_ml'].astype('float')
calc_df['mass_storage_tube_only_g'] = calc_df['mass_storage_tube_only_g'].astype('float')
calc_df['density_sample_g_ml'] = calc_df['density_sample_g_ml'].astype('float')
calc_df['density_sample_kg_l'] = calc_df['density_sample_kg_l'].astype('float')

In [ ]:
calc_df.head()

In [ ]:
# Check that all values required for abs quant calculations are present
slurry_calc_columns = ['vol_homogenate_aliquot_input_ul', 'vol_homogenate_aliquot_input_ml', 'mass_storage_tube_and_storage_liquid_after_sample_g', 'mass_storage_tube_and_storage_liquid_after_sample_mg', 'mass_storage_tube_and_storage_liquid_before_sample_g', 'mass_storage_tube_and_storage_liquid_before_sample_mg', 'mass_storage_tube_only_g', 'density_storage_liquid_kg_l', 'density_storage_liquid_g_ml', 'storage_liquid_lot_number', 'density_sample_g_ml', 'density_sample_kg_l', 'storage_liquid_type']

if set(slurry_calc_columns).issubset(calc_df.columns):
    print('All abs quant column requirements for slurry sample calculations have been met. Proceed.')
else:
    print('All abs quant column requirements for slurry sample calculations have NOT been met. DO NOT proceed, and determine what needs to be added.')

In [ ]:
# Step 3: Generate calculated values and add new columns to calc_df:

In [ ]:
# Calcuate and generate columns for mass of sample in storage tube and mass of the storage liquid
calc_df = calc_df.assign(calc_mass_sample_in_storage_tube_mg=calc_df['mass_storage_tube_and_storage_liquid_after_sample_mg']-calc_df['mass_storage_tube_and_storage_liquid_before_sample_mg'],
                         calc_mass_sample_in_storage_tube_g=calc_df['mass_storage_tube_and_storage_liquid_after_sample_g']-calc_df['mass_storage_tube_and_storage_liquid_before_sample_g'],
                         calc_mass_storage_liquid_only_g=calc_df['mass_storage_tube_and_storage_liquid_before_sample_g']-calc_df['mass_storage_tube_only_g'])

In [ ]:
# Calcuate and generate columns for volume of storage liquid only and volume of sample in storage tube
# In different cell because these equations contain columns that were defined above - any way to optimize this?
calc_df = calc_df.assign(calc_vol_storage_liquid_only_ml=calc_df['calc_mass_storage_liquid_only_g']/calc_df['density_storage_liquid_g_ml'],
                         calc_vol_sample_in_storage_tube_ml=(calc_df['mass_storage_tube_and_storage_liquid_after_sample_g']-calc_df['mass_storage_tube_and_storage_liquid_before_sample_g'])/calc_df['density_sample_g_ml'])

In [ ]:
# Calcuate and generate columns for volume homogenate in storage tube
calc_df = calc_df.assign(calc_vol_homogenate_in_storage_tube_ml=calc_df['calc_vol_storage_liquid_only_ml']+calc_df['calc_vol_sample_in_storage_tube_ml'])

In [ ]:
# Calcuate and generate columns for density of homogenate
calc_df = calc_df.assign(calc_density_homogenate_g_ml=(calc_df['mass_storage_tube_and_storage_liquid_after_sample_g']-calc_df['mass_storage_tube_only_g'])/calc_df['calc_vol_homogenate_in_storage_tube_ml'])

In [ ]:
# Calcuate and generate columns for mass of the homogenate aliquotted into the matrix tube and mass of the sample aliquotted into the matrix tube
calc_df = calc_df.assign(calc_mass_homogenate_aliquot_input_g=calc_df['calc_density_homogenate_g_ml']*calc_df['vol_homogenate_aliquot_input_ml'],
                         calc_mass_sample_aliquot_input_g=((calc_df['density_sample_g_ml']*calc_df['density_storage_liquid_g_ml']*calc_df['vol_homogenate_aliquot_input_ml'])-(calc_df['density_sample_g_ml']*calc_df['calc_density_homogenate_g_ml']*calc_df['vol_homogenate_aliquot_input_ml']))/(calc_df['density_storage_liquid_g_ml']-calc_df['density_sample_g_ml']))

In [ ]:
# Calcuate and generate columns for mg of sample alituotted into matrix tube as mass of the storage liquid aliquotted into the matrix tube
calc_df = calc_df.assign(calc_mass_sample_aliquot_input_mg=calc_df['calc_mass_sample_aliquot_input_g']*1000,
                         calc_mass_storage_liquid_aliquot_input_g=calc_df['calc_mass_homogenate_aliquot_input_g']-calc_df['calc_mass_sample_aliquot_input_g'])

In [ ]:
# Step 4: Check output is as expected and export .csv:

In [ ]:
# Check new columns are present and expected
calc_df.head()

In [ ]:
# Export .txt
calc_df.to_csv('/Users/mmbryant/Documents/Python Things/Initial Attempts/nph_calc_from_core_NOTEBOOK_test.txt', sep='\t', index=False)